In [15]:
import sys
sys.path.insert(0, "/home/alexp/NBA_Models/src/scripts/")

In [16]:
from data.transform_data import transform
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlite3
import tensorflow as tf

%load_ext tensorboard

import datetime

pd.set_option("display.max_columns",None)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [72]:
path = "/home/alexp/NBA_Models/sqlite/db/nba_data.db"
conn = sqlite3.connect(path)

t = transform(conn,2010,2020)
clean = t.transform(type='agg_boxscores')
scaler = t.standard_scalar(clean)
lable_enc = t.label_encoder(scaler,['SEASON','TEAM_ABBREVIATION'])
train_data = lable_enc
#train_data = lable_enc.drop(columns=['TEAM_ID','TEAM_NAME','GAME_ID','GAME_DATE','MATCHUP'])

test = transform(conn,2021,2023)
test_clean = test.transform(type='agg_boxscores')
test_scaler = test.standard_scalar(test_clean)
test_lable_enc = test.label_encoder(test_scaler,['SEASON','TEAM_ABBREVIATION'])
test_data = test_lable_enc


In [59]:
train_data

,SEASON,TEAM_ABBREVIATION,HOME_GAME,WL,FG2M,FG2A,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,E_NET_RATING,NET_RATING,POSS,PIE,PTS_2PT_MR,PTS_FB,PTS_OFF_TOV,PTS_PAINT,AST_2PM,AST_3PM,UAST_2PM,UAST_3PM
2,0,12,0,0,0.729863,0.482261,-0.558035,-0.930782,-0.747292,0.015774,-0.069565,-0.672908,-0.597883,0.733279,1.141185,-0.349744,0.456749,0.145184,-0.275042,-0.922977,103.9,105.1,109.1,117.2,-5.2,-12.1,98.0,0.497,0.721859,0.842448,1.494346,0.238555,1.057119,-0.334230,-0.241490,-0.364551
3,0,13,1,1,-0.220879,0.704900,0.931256,0.034125,0.081879,0.691828,2.071226,0.060553,1.227839,-0.040645,0.118592,0.449395,1.214670,0.606899,0.725374,0.922961,109.1,117.2,103.9,105.1,5.2,12.1,99.0,0.503,0.168028,-0.077681,1.821529,-0.356668,-0.532157,0.817254,0.232991,0.259013
4,0,21,0,0,-0.411028,1.818091,-0.309820,-1.172008,-1.908133,-1.741968,0.733232,-1.406370,-0.750026,-1.201530,0.800321,0.449395,1.214670,1.299472,-1.506324,-0.709984,84.1,92.6,101.0,103.2,-16.8,-10.6,94.0,0.339,0.998774,-0.384391,0.676390,-1.051096,-1.213275,-0.334230,0.707471,-0.364551
5,0,11,1,1,-0.411028,-0.408292,-0.806251,-1.413235,0.745217,1.232672,-0.069565,0.060553,0.010691,-1.201530,-1.244865,5.244230,1.719950,-1.701676,-0.736773,0.709968,101.0,103.2,84.1,92.6,16.8,10.6,94.0,0.661,0.721859,-0.077681,2.148712,-0.951892,-0.759196,-1.197843,0.232991,0.259013
6,0,29,0,0,-0.981473,-0.519611,-0.061605,-0.448329,1.076886,1.232672,0.733232,-1.589735,-0.902170,-0.040645,-0.904001,-0.749313,0.709389,2.222902,-0.351997,-0.780982,106.2,104.1,113.5,115.3,-7.4,-11.2,98.0,0.413,0.860316,-0.691100,1.003572,-1.646319,-0.305118,0.529383,-0.953210,-0.988114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20367,7,16,1,1,0.539715,1.150176,-0.806251,0.034125,1.076886,0.827039,1.803627,-0.489543,0.619265,-0.234125,1.141185,-0.749313,-2.322296,-0.547389,0.186688,0.425977,111.5,112.4,104.7,106.2,6.8,6.2,97.0,0.536,0.444943,0.382383,1.167164,0.238555,0.376001,-0.909972,0.470231,-0.364551
20368,7,23,1,0,1.110160,1.150176,0.683041,-1.172008,-1.244796,-1.606757,-0.604763,-1.223004,-1.358600,-0.040645,0.459456,0.049826,-1.311734,-0.085674,0.956240,-0.283999,128.2,129.3,128.8,135.2,-0.6,-5.8,92.0,0.503,0.583401,-0.077681,0.185616,0.734575,-0.078078,-0.046359,1.656431,1.506140
20369,7,16,0,1,1.300308,0.148304,0.931256,-0.086489,-1.410630,-0.795492,0.198034,-1.406370,-1.054313,0.539798,-0.222272,-1.548453,-0.553813,-0.778246,1.264060,0.283982,128.8,135.2,128.2,129.3,0.6,5.8,91.0,0.497,1.691062,1.302512,-0.141567,0.139351,-0.078078,0.817254,1.656431,0.259013
20370,7,16,1,1,0.349566,-0.296972,-1.054466,-0.207102,1.242720,0.827039,0.198034,1.527476,1.379983,-0.621087,0.800321,0.449395,1.214670,-0.778246,-0.121132,0.496975,102.2,101.9,94.8,96.1,7.4,5.9,103.0,0.583,-0.247344,-0.231036,-1.123115,0.436963,0.376001,-1.485714,-0.004250,0.882576


# Create Input Array

In [60]:
def create_windows(data,window_size):
  X_list = []
  y_list = []

  for i in range(len(data['TEAM_ABBREVIATION'].unique())):
      team = data.loc[data['TEAM_ABBREVIATION'] == i]
      team_label = team['WL'].copy()
      team = team.drop('WL',axis=1)

      team_np = team.to_numpy()
      team_label = team_label.to_numpy()

      #temp_X = []
      #temp_y = []

      for i in range(len(team_np)-window_size):        
        row = [a for a in team_np[i:i+window_size]]
        X_list.append(row)

        label = team_label[i+window_size]
        y_list.append(label)

  X_list = np.array(X_list)
  y_list = np.array(y_list)

  return X_list,y_list

In [61]:
X_list,y_list = create_windows(train_data,30)
#X_list,y_list = train_data.loc[:,train_data.columns != 'WL'],train_data['WL']
#X_test_list,y_test_list = test_data.loc[:,test_data.columns != 'WL'],test_data['WL']
X_list.shape

(16518, 30, 35)

# Model

In [62]:
from model import ModelConfig

In [63]:
model_config = ModelConfig(X_list,y_list,[.75,.15,.10])
X_train,y_train,X_test,y_test,X_val,y_val = model_config.train_test_val()

In [64]:
import keras
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.losses import MeanSquaredError,BinaryCrossentropy
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.metrics import RootMeanSquaredError, Accuracy
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

#,kernel_regularizer=regularizers.L2(0.0001)
model = model_config.create_sequential()
model.add(LSTM(256,'tanh',kernel_regularizer=regularizers.L2(0.0001)))
model.add(Dropout(.2))
model.add(Dense(64,'tanh',kernel_regularizer=regularizers.L2(0.0001)))
model.add(Dropout(.2))
model.add(Dense(32,'tanh',kernel_regularizer=regularizers.L2(0.0001)))
model.add(Dropout(.2))
model.add(Dense(1,'tanh'))

model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=.0001), metrics=[RootMeanSquaredError(),'accuracy'])

In [65]:
keras.backend.clear_session()

In [66]:
cp = ModelCheckpoint("/home/alexp/NBA_Models/models/models/model5", save_best_only=True)
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=100,callbacks=[tensorboard_callback,cp])

Epoch 1/100
381/388 [============================>.] - ETA: 0s - loss: 0.3195 - root_mean_squared_error: 0.5495 - accuracy: 0.5266INFO:tensorflow:Assets written to: /home/alexp/NBA_Models/models/models/model5/assets


INFO:tensorflow:Assets written to: /home/alexp/NBA_Models/models/models/model5/assets


388/388 [==============================] - 5s 9ms/step - loss: 0.3197 - root_mean_squared_error: 0.5497 - accuracy: 0.5263 - val_loss: 0.2776 - val_root_mean_squared_error: 0.5128 - val_accuracy: 0.5418
Epoch 2/100
380/388 [============================>.] - ETA: 0s - loss: 0.2978 - root_mean_squared_error: 0.5333 - accuracy: 0.5288INFO:tensorflow:Assets written to: /home/alexp/NBA_Models/models/models/model5/assets


INFO:tensorflow:Assets written to: /home/alexp/NBA_Models/models/models/model5/assets


388/388 [==============================] - 3s 8ms/step - loss: 0.2975 - root_mean_squared_error: 0.5330 - accuracy: 0.5287 - val_loss: 0.2616 - val_root_mean_squared_error: 0.4992 - val_accuracy: 0.5696
Epoch 3/100
381/388 [============================>.] - ETA: 0s - loss: 0.2827 - root_mean_squared_error: 0.5204 - accuracy: 0.5397INFO:tensorflow:Assets written to: /home/alexp/NBA_Models/models/models/model5/assets


...

388/388 [==============================] - 3s 9ms/step - loss: 0.2394 - root_mean_squared_error: 0.4862 - accuracy: 0.5965 - val_loss: 0.2403 - val_root_mean_squared_error: 0.4872 - val_accuracy: 0.5920
Epoch 67/100
388/388 [==============================] - 2s 5ms/step - loss: 0.2391 - root_mean_squared_error: 0.4859 - accuracy: 0.5953 - val_loss: 0.2408 - val_root_mean_squared_error: 0.4877 - val_accuracy: 0.5987
Epoch 68/100
388/388 [==============================] - 2s 5ms/step - loss: 0.2386 - root_mean_squared_error: 0.4854 - accuracy: 0.5993 - val_loss: 0.2407 - val_root_mean_squared_error: 0.4876 - val_accuracy: 0.5902
Epoch 69/100
388/388 [==============================] - 2s 5ms/step - loss: 0.2387 - root_mean_squared_error: 0.4856 - accuracy: 0.6009 - val_loss: 0.2409 - val_root_mean_squared_error: 0.4878 - val_accuracy: 0.5969
Epoch 70/100
388/388 [==============================] - 2s 6ms/step - loss: 0.2390 - root_mean_squared_error: 0.4859 - accuracy: 0.5961 - val_loss: 0

INFO:tensorflow:Assets written to: /home/alexp/NBA_Models/models/models/model5/assets


388/388 [==============================] - 3s 9ms/step - loss: 0.2385 - root_mean_squared_error: 0.4855 - accuracy: 0.5986 - val_loss: 0.2399 - val_root_mean_squared_error: 0.4869 - val_accuracy: 0.6023
Epoch 73/100
388/388 [==============================] - 2s 5ms/step - loss: 0.2389 - root_mean_squared_error: 0.4859 - accuracy: 0.5968 - val_loss: 0.2410 - val_root_mean_squared_error: 0.4881 - val_accuracy: 0.5944
Epoch 74/100
388/388 [==============================] - 2s 6ms/step - loss: 0.2384 - root_mean_squared_error: 0.4855 - accuracy: 0.6014 - val_loss: 0.2421 - val_root_mean_squared_error: 0.4893 - val_accuracy: 0.5920
Epoch 75/100
388/388 [==============================] - 2s 5ms/step - loss: 0.2381 - root_mean_squared_error: 0.4851 - accuracy: 0.6000 - val_loss: 0.2409 - val_root_mean_squared_error: 0.4880 - val_accuracy: 0.5926
Epoch 76/100
388/388 [==============================] - 2s 6ms/step - loss: 0.2383 - root_mean_squared_error: 0.4853 - accuracy: 0.5947 - val_loss: 0

INFO:tensorflow:Assets written to: /home/alexp/NBA_Models/models/models/model5/assets


388/388 [==============================] - 3s 8ms/step - loss: 0.2374 - root_mean_squared_error: 0.4846 - accuracy: 0.6001 - val_loss: 0.2397 - val_root_mean_squared_error: 0.4870 - val_accuracy: 0.5908
Epoch 87/100
388/388 [==============================] - 2s 5ms/step - loss: 0.2378 - root_mean_squared_error: 0.4850 - accuracy: 0.6000 - val_loss: 0.2401 - val_root_mean_squared_error: 0.4874 - val_accuracy: 0.5969
Epoch 88/100
388/388 [==============================] - 2s 6ms/step - loss: 0.2375 - root_mean_squared_error: 0.4847 - accuracy: 0.6015 - val_loss: 0.2408 - val_root_mean_squared_error: 0.4881 - val_accuracy: 0.5956
Epoch 89/100
388/388 [==============================] - 2s 5ms/step - loss: 0.2377 - root_mean_squared_error: 0.4850 - accuracy: 0.5991 - val_loss: 0.2430 - val_root_mean_squared_error: 0.4904 - val_accuracy: 0.5878
Epoch 90/100
388/388 [==============================] - 2s 5ms/step - loss: 0.2375 - root_mean_squared_error: 0.4847 - accuracy: 0.5999 - val_loss: 0

In [67]:
from tensorflow.keras.models import load_model
model = load_model("/home/alexp/NBA_Models/models/models/model5")

In [68]:
model.evaluate(X_test,y_test)

78/78 [==============================] - 0s 3ms/step - loss: 0.2400 - root_mean_squared_error: 0.4873 - accuracy: 0.5892


[0.240028977394104, 0.4873332381248474, 0.5891848206520081]

In [69]:
preds_raw = model.predict(X_test)
actual = y_test
preds = []
for i,pred in enumerate(preds_raw):
    if pred[0] >=.5:
        preds.append(1)
    elif pred[0] <.5:
        preds.append(0)

78/78 [==============================] - 0s 1ms/step


In [70]:
confusion_matrix(actual,preds)

array([[731, 488],
       [530, 729]])

In [71]:
accuracy_score(actual,preds), f1_score(actual,preds)

(0.5891848264729621, 0.5888529886914378)

In [40]:
from data.load_data import load_clean

In [42]:
load = load_clean("/home/alexp/NBA_Models/sqlite/db/nba_data.db")
load.agg_boxscores_raw()

,SEASON,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PTS_DIFF,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,E_NET_RATING,NET_RATING,AST_PCT,AST_TOV,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,E_TM_TOV_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE,PCT_FGA_2PT,PCT_FGA_3PT,PCT_PTS_2PT,PCT_PTS_2PT_MR,PCT_PTS_3PT,PCT_PTS_FB,PCT_PTS_FT,PCT_PTS_OFF_TOV,PCT_PTS_PAINT,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,PCT_AST_FGM,PCT_UAST_FGM
0,2013-14,1610612748,MIA,Miami Heat,0021300002,2013-10-29,MIA vs. CHI,W,240,37,72,0.514,11,20,0.550,22,29,0.759,5,35,40,26,10,7,20,21,107,12,107.3,108.1,93.9,96.9,13.3,11.1,0.703,1.30,19.9,NaN,0.706,NaN,20.048,20.2,0.590,0.631,1.0,0.197,100.44,98.5,82.08,99,0.599,0.722,0.278,0.486,0.168,0.308,0.093,0.206,0.140,0.318,0.577,0.423,1.000,0.000,0.703,0.297
1,2013-14,1610612741,CHI,Chicago Bulls,0021300002,2013-10-29,CHI @ MIA,L,240,35,83,0.422,7,26,0.269,18,23,0.783,11,30,41,23,11,4,19,27,95,-12,93.9,96.9,107.3,108.1,-13.3,-11.1,0.657,1.21,17.0,0.294,NaN,NaN,18.790,19.4,0.464,0.510,1.0,0.198,100.44,98.5,82.08,98,0.401,0.687,0.313,0.589,0.063,0.221,0.179,0.189,0.316,0.526,0.571,0.429,1.000,0.000,0.657,0.343
2,2013-14,1610612746,LAC,Los Angeles Clippers,0021300003,2013-10-29,LAC @ LAL,L,240,41,83,0.494,8,21,0.381,13,23,0.565,10,30,40,27,11,4,16,21,103,-13,103.9,105.1,109.1,117.2,-5.2,-12.1,0.659,1.69,19.8,0.255,0.561,0.423,16.142,16.3,0.542,0.553,1.0,0.200,102.72,98.5,82.08,98,0.497,0.747,0.253,0.641,0.175,0.233,0.184,0.126,0.243,0.466,0.606,0.394,0.875,0.125,0.659,0.341
3,2013-14,1610612747,LAL,Los Angeles Lakers,0021300003,2013-10-29,LAL vs. LAC,W,240,42,93,0.452,14,29,0.483,18,28,0.643,18,34,52,23,8,6,19,23,116,13,109.1,117.2,103.9,105.1,5.2,12.1,0.548,1.21,15.6,0.439,0.745,0.577,17.871,19.2,0.527,0.551,1.0,0.195,102.72,98.5,82.08,99,0.503,0.688,0.312,0.483,0.121,0.362,0.112,0.155,0.241,0.362,0.429,0.571,0.786,0.214,0.548,0.452
4,2013-14,1610612753,ORL,Orlando Magic,0021300001,2013-10-29,ORL @ IND,L,240,36,93,0.387,9,19,0.474,6,10,0.600,13,26,39,17,10,6,19,26,87,-10,84.1,92.6,101.0,103.2,-16.8,-10.6,0.472,0.89,12.7,0.383,0.683,0.505,18.375,20.2,0.435,0.447,1.0,0.194,99.74,94.0,78.33,94,0.339,0.796,0.204,0.621,0.230,0.310,0.115,0.069,0.241,0.391,0.370,0.630,0.778,0.222,0.472,0.528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26663,2023-24,1610612766,CHA,Charlotte Hornets,0022300484,2024-01-05,CHA @ CHI,L,240,36,89,0.404,10,34,0.294,9,12,0.750,12,30,42,20,7,1,13,20,91,-13,95.5,97.8,112.5,111.8,-16.9,-14.0,0.556,1.54,15.7,0.273,0.865,0.511,13.644,14.0,0.461,0.483,1.0,0.194,93.88,93.0,77.50,93,0.367,0.618,0.382,0.571,0.176,0.330,0.110,0.099,0.176,0.396,0.423,0.577,0.900,0.100,0.556,0.444
26664,2023-24,1610612740,NOP,New Orleans Pelicans,0022300486,2024-01-05,NOP vs. LAC,L,240,33,92,0.359,12,29,0.414,17,23,0.739,14,29,43,22,9,5,9,17,95,-16,97.8,101.1,115.9,118.1,-18.0,-17.0,0.667,2.44,16.5,0.328,0.682,0.476,9.267,9.6,0.424,0.465,1.0,0.202,96.46,94.0,78.33,94,0.391,0.685,0.315,0.442,0.042,0.379,0.116,0.179,0.189,0.400,0.619,0.381,0.750,0.250,0.667,0.333
26665,2023-24,1610612746,LAC,LA Clippers,0022300486,2024-01-05,LAC @ NOP,W,240,40,82,0.488,15,36,0.417,16,20,0.800,10,40,50,28,6,7,15,18,111,16,115.9,118.1,97.8,101.1,18.0,17.0,0.700,1.87,20.9,0.318,0.672,0.524,15.658,16.0,0.579,0.611,1.0,0.198,96.46,94.0,78.33,94,0.609,0.561,0.439,0.450,0.090,0.405,0.117,0.144,0.072,0.360,0.640,0.360,0.800,0.200,0.700,0.300
26666,2023-24,1610612750,MIN,Minnesota Timberwolves,0022300485,2024-01-05,MIN @ HOU,W,240,46,88,0.523,14,33,0.424,16,20,0.800,9,43,52,31,12,7,14,18,122,27,119.8,119.6,90.8,95.0,29.0,24.6,0.674,2.21,21.9,0.273,0.821,0.580,13.752,13.7,0.602,0.630,1.0,0.1